# Setup

In [4]:
%load_ext autoreload
%autoreload 2
#!pip install sb3-contrib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from sb3_contrib import MaskablePPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor
import time
from collections import deque
from stable_baselines3.common.vec_env import SubprocVecEnv
from math import exp, log
import random

In [6]:
class TimerCallback(BaseCallback):
    def __init__(self):
        super().__init__()
        self.start_time = time.time()
    
    def _on_step(self):
        if self.num_timesteps % 1000 == 0:
            elapsed = time.time() - self.start_time
            rate = self.num_timesteps / elapsed
            remaining = (self.locals['total_timesteps'] - self.num_timesteps) / rate
            print(f"Step {self.num_timesteps}, {elapsed:.0f}s elapsed, {remaining:.0f}s remaining")
        return True

class GradNormCallback(BaseCallback):
    def _on_step(self):
        if hasattr(self.model.policy, 'parameters'):
            total_norm = 0
            for p in self.model.policy.parameters():
                if p.grad is not None:
                    param_norm = p.grad.data.norm(2)
                    total_norm += param_norm.item() ** 2
            total_norm = total_norm ** (1. / 2)
            
            self.logger.record("train/grad_norm", total_norm)
        return True

from stable_baselines3.common.callbacks import BaseCallback

# up the difficulty upon reaching certain (rolling) mean episode length thresholds
class CurriculumCallback(BaseCallback):
    def __init__(self, env, verbose = 1):
        super().__init__(verbose = verbose)
        self.env = env
        self.verbose = verbose
        self.thresholds = [7.5, 5] # episode length required before upping difficulty
        self.current_stage = 0
        self.ep_lengths = deque(maxlen=30)

    def _on_step(self):
        infos = self.locals.get("infos", [])
        for info in infos:
            if "episode" not in info:
                continue
                
            ep_len = info["episode"]["l"]
            self.ep_lengths.append(ep_len)
            avg_len = sum(self.ep_lengths) / len(self.ep_lengths)
            if self.current_stage < len(self.thresholds) and avg_len > self.thresholds[self.current_stage]:
                self.current_stage += 1
                self.env.set_difficulty(self.current_stage)
                if self.verbose:
                    print(f"Average episode length: {avg_len:.2f} — Switched to difficulty: {self.current_stage}")
        return True

In [7]:
from apad_env import APADEnv

In [8]:
def make_env(mo=None, day=None):
   if mo is None or day is None:
       env = APADEnv()  # random board
   else:
       env = APADEnv(mo, day)  # fixed board
   return Monitor(env)

# Training strategy: repeat boards + increased batch size

In [9]:
# Repeat boards setup
n_boards = 100
board_configs = []
for _ in range(n_boards):
   mo = random.randint(1, 12)
   day = random.randint(1, 30)
   board_configs.append((mo, day))

In [10]:
selected_boards = random.sample(board_configs, 8)
env = SubprocVecEnv([lambda mo=mo, day=day: make_env(mo, day) for mo, day in selected_boards])
model = MaskablePPO(
    "MlpPolicy",
    env,
    tensorboard_log="./maskable_ppo_logs_16/",
    #ent_coef= ent_coef_i,
    #learning_rate=0.0004,
    verbose=1,
    batch_size=1000,
    n_steps=2048*2,
    # gamma=0.999,
)

total_timesteps = 1000000
checkpoint_interval = 50000
for i in range(0, total_timesteps, checkpoint_interval):
    reset = True if i == 0 else False
    remaining_steps = min(checkpoint_interval, total_timesteps - i)
    progress = i / total_timesteps

    # new boards to train on
    selected_boards = random.sample(board_configs, 8)
    env = SubprocVecEnv([lambda mo=mo, day=day: make_env(mo, day) for mo, day in selected_boards])
    model.set_env(env)
    print(f"Switching to boards: {selected_boards}")
    
    model.learn(total_timesteps=remaining_steps, reset_num_timesteps=reset, callback=[TimerCallback(),GradNormCallback()])
    
    if i % 1000000 == 0:
        model.save(f"mppo_model_v1.3_{i}")

model.save(f"mppo_model_v1.3")

Using cpu device
Switching to boards: [(12, 5), (4, 27), (11, 1), (9, 17), (3, 14), (11, 14), (8, 10), (6, 6)]
Logging to ./maskable_ppo_logs_16/PPO_5
Step 1000, 6s elapsed, 295s remaining
Step 2000, 12s elapsed, 279s remaining
Step 3000, 18s elapsed, 286s remaining
Step 4000, 307s elapsed, 3534s remaining
Step 5000, 407s elapsed, 3661s remaining
Step 6000, 414s elapsed, 3036s remaining
Step 7000, 434s elapsed, 2666s remaining
Step 8000, 441s elapsed, 2315s remaining
Step 9000, 448s elapsed, 2040s remaining
Step 10000, 5284s elapsed, 21137s remaining
Step 11000, 5291s elapsed, 18760s remaining
Step 12000, 7276s elapsed, 23041s remaining
Step 13000, 8590s elapsed, 24449s remaining
Step 14000, 8598s elapsed, 22109s remaining
Step 15000, 9015s elapsed, 21034s remaining
Step 16000, 10940s elapsed, 23247s remaining
Step 17000, 14599s elapsed, 28339s remaining
Step 18000, 18254s elapsed, 32452s remaining
Step 19000, 18262s elapsed, 29796s remaining
Step 20000, 21918s elapsed, 32876s remainin

KeyboardInterrupt: 

# Old

## Training Strategy: entropy rampdown + repeat boards + increased batch size

In [ ]:
# Entropy schedule constants
ent_coef_i = 0.2
ent_coef_f = 0.015
D = log(ent_coef_i/ent_coef_f)

In [ ]:
# Repeat boards setup
n_boards = 100
board_configs = []
for _ in range(n_boards):
   mo = random.randint(1, 12)
   day = random.randint(1, 30)
   board_configs.append((mo, day))

In [ ]:
#current_mo, current_day = board_configs[0]
#env = SubprocVecEnv([lambda mo=current_mo, day=current_day: make_env(mo, day) for _ in range(8)])
selected_boards = random.sample(board_configs, 8)
env = SubprocVecEnv([lambda mo=mo, day=day: make_env(mo, day) for mo, day in selected_boards])
model = MaskablePPO(
    "MlpPolicy",
    env,
    tensorboard_log="./maskable_ppo_logs_16/",
    ent_coef= ent_coef_i,
    #learning_rate=0.0004,
    verbose=1,
    batch_size=1000,
    #gamma=0.999,
)

total_timesteps = 3000000
checkpoint_interval = 50000
for i in range(0, total_timesteps, checkpoint_interval):
    reset = True if i == 0 else False
    remaining_steps = min(checkpoint_interval, total_timesteps - i)
    progress = i / total_timesteps

    # new boards to train on
    selected_boards = random.sample(board_configs, 8)
    env = SubprocVecEnv([lambda mo=mo, day=day: make_env(mo, day) for mo, day in selected_boards])
    model.set_env(env)
    print(f"Switching to boards: {selected_boards}")
   
    if progress < 0.9:
        model.ent_coef = ent_coef_i * exp(-1*D*progress)
    
    print(f"ENTCOEF {model.ent_coef}")
    
    model.learn(total_timesteps=remaining_steps, reset_num_timesteps=reset, callback=[TimerCallback(),GradNormCallback()])
    
    if i % 1000000 == 0:
        model.save(f"mppo_model_v1.2_{i}")

model.save(f"mppo_model_v1.2")

## Training strategy: incremental difficulty rampup

### step 1: no date, 6 pieces = win
v0

In [ ]:
env = APADEnv(-1,-1,2)
model = None
model = MaskablePPO(
    "MlpPolicy",
    env,
    tensorboard_log="./maskable_ppo_logs_14/",
    ent_coef=0.1,
    learning_rate=0.003,
    verbose=1,
)
model.learn(total_timesteps=75000, callback=[TimerCallback(), GradNormCallback()])
model.save(f"mppo_model_v0_2")

### step 2:

a) no date, 7 pieces

v0a


In [ ]:
model = MaskablePPO.load("mppo_model_v0")
env = APADEnv(-1,-1,1)
model.set_env(env)
model.learn(100000, reset_num_timesteps=True)
model.save(f"mppo_model_v0a")

b) day only, 6 pieces

v0b

In [ ]:
model = MaskablePPO.load("mppo_model_v0")
env = APADEnv(-1,None,2)
model.set_env(env)
model.learn(100000, reset_num_timesteps=True)
model.save(f"mppo_model_v0b")

c) month only, 6 pieces

v0c

In [ ]:
model = MaskablePPO.load("mppo_model_v0")
env = APADEnv(None,-1,2)
model.set_env(env)
model.learn(100000, reset_num_timesteps=True)
model.save(f"mppo_model_v0c")

## Training strategy: Vanilla

In [ ]:
#total_timesteps = 100000
#checkpoint_interval = 25000
#for i in range(0, total_timesteps, checkpoint_interval):
#    remaining_steps = min(checkpoint_interval, total_timesteps - i)
#    if i == 0:
#        model.learn(total_timesteps=remaining_steps, reset_num_timesteps=True, callback=[TimerCallback(),GradNormCallback()])
#    else:
#        model.learn(total_timesteps=remaining_steps, reset_num_timesteps=False, callback=[TimerCallback(),GradNormCallback()])
#

In [ ]:
env = APADEnv(-1)
#env.reset()
model = None
model = MaskablePPO(
    "MlpPolicy",
    env,
    tensorboard_log="./maskable_ppo_logs_10/",
    ent_coef=0.03,
    verbose=1,
)
total_timesteps = 300000
checkpoint_interval = 50000
for i in range(0, total_timesteps, checkpoint_interval):
    remaining_steps = min(checkpoint_interval, total_timesteps - i)
    if i == 0:
        model.learn(total_timesteps=remaining_steps, reset_num_timesteps=True, callback=[TimerCallback(),GradNormCallback()])
    else:
        model.learn(total_timesteps=remaining_steps, reset_num_timesteps=False, callback=[TimerCallback(),GradNormCallback()])
    model.save(f"mppo_model_{11}")

In [ ]:
env = APADEnv(-1,-1)
env.reset()
model = None
model = MaskablePPO(
    "MlpPolicy",
    env,
    #n_steps = 512,
    tensorboard_log="./maskable_ppo_logs_9/",
    verbose=1,
)
model.learn(total_timesteps=50000, reset_num_timesteps=False, callback=[TimerCallback(), GradNormCallback()])
model.save(f"mppo_model_9")

In [ ]:
for i in range(4):
    env.reset()
    model = None
    model = MaskablePPO(
        "MlpPolicy",
        env,
        #n_steps = 512,
        tensorboard_log="./maskable_ppo_logs_9/",
        verbose=1,
    )
    model.learn(total_timesteps=50000, reset_num_timesteps=True, callback=[TimerCallback(), GradNormCallback()])
    model.save(f"mppo_model_{i}")

In [ ]:
model.save("mppo_model_25k_2025-06-19_1500")